Goal is to translate data from Pixelav (which comes in the form of 8x32 array) into something the board can understand (string of bits 00, 01, 10, 11 that corresponds to different charge thresholds). Then want to map it into array1.
\
\
**questions:**
- what are the different arrays 0,1,2 / what are we planning on filling them with 
- how do you take a time stamp? i don't actually know what pixelav is putting out so i don't know where that comes from

pixelav: 0:3 y, 0:15 x (8x32 --> 256 pixels). then each pixel goes to 2 bits which classify its charge. 

**mapping the (fake) pixelav output onto the ideal size (13x21-->8x32).** there are a couple ways to do this. we could take all the non-zero cluster information and just stick it in the center of the 8x32 array, but we aren't sure yet how/if this translation would affect the ultimate results. for now, i'm going to just truncate the values from the top and bottom rows (2 from the top and 3 from the bottom) and pad the extra columns of the pixelav output with zeros so that it fits into the 8x32 framework. i will also work in some "overflow" in case we accidentally chop off a part of a cluster, where i will add any data from the "chopped" pixels to the first row that we keep

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
final_size = (8,32)
thresh0 = 400
thresh1 = 1600
thresh2 = 2400

In [3]:
def reshape_pixelav(cluster, cluster_id):
    """
    Inputs: 13x21 array from Pixelav, labelled with the time slice
    Outputs: 8x32 array with (hopefully) the same data which can be fed into Array1
    
    Truncates original rows on top and bottom and adds zeros to the extra columns on the right in order to fit 8x32 shape. 
    To avoid accidentally truncating data, we add the values of each deleted pixel to the nearest row. For example, since 
    cut off row 12, if there were any charge in that row it would be added to the same column in row 9, which is the last
    column we save. A message will appear if this happens.
    """
    # truncating and overflow
    pixelav_row = np.shape(cluster)[0]
    pixelav_col = np.shape(cluster)[1]
    chop_start = 1
    chop_stop = 9
    overflow_top = np.zeros(pixelav_col)
    overflow_bottom = np.zeros(pixelav_col)
    for col in range(pixelav_col):
        for row in range(chop_start):
            overflow_top[col] += cluster[row, col]
        for row in range(chop_stop + 1, pixelav_row):
            overflow_bottom[col] += cluster[row, col]
    cluster[chop_start, :] += overflow_top
    cluster[chop_stop, :] += overflow_bottom
    chop_cluster = cluster[chop_start:chop_stop, :]

    if np.any(overflow_top) or np.any(overflow_bottom):
        print(f"Overflow detected at {cluster_id}ns:")
        if np.any(overflow_top):
            print(f"  Top overflow: {overflow_top}")
        if np.any(overflow_bottom):
            print(f"  Bottom overflow: {overflow_bottom}")     

    #padding
    final_input = np.zeros(final_size)
    final_input[:, :chop_cluster.shape[1]] = chop_cluster
    return final_input

In [4]:
def make_bit_string(final_input):
    """
    Inputs: Reshaped 8x32 array of Pixelav data
    Outputs: String of bits in (hopefully) correct order, ready to go into Array 1
    """
    big_row_start= 0
    pixel_data = []
    while big_row_start < 8:
        for column in reversed(range(32)):
            for mini_row in range(2):
                row = mini_row + big_row_start
                charge = final_input[row, column]
                if charge <= thresh0: 
                    label = "00"
                elif thresh0 < charge < thresh1:
                    label = "01"
                elif thresh1 < charge < thresh2:
                    label = "10"
                else:
                    label = "11"
                pixel_data.append(label)
        big_row_start += 2
    bit_string = ''.join(reversed(pixel_data))
    return bit_string

In [5]:
def make_array1(bit_string):
    ar1row = 256
    ar1col = 16
    array1 = np.zeros((ar1row,ar1col))
    len_shiftreg = (68+1)*16+4
    len_bits = len(bit_string)
    len_dnn = (256-100)*16+4*3

    # fake numbers for shift register and dnn parts of the array
    shiftreg = [0]*len_shiftreg
    dnn = [0]*len_dnn

    i=0
    for row in range(ar1row):
        for col in reversed(range(ar1col)):
            if i < len_shiftreg:
                array1[row,col] = shiftreg[i]
            elif len_shiftreg <= i < len_shiftreg + len_bits:
                array1[row, col] = bit_string[i - len_shiftreg]
            elif i >= len_shiftreg+len_bits:
                array1[row, col] = dnn[i - (len_shiftreg+len_bits)]
            i += 1
    return array1

In [6]:
# using eliza's BIB parquet file as a sample pixelav output
truth = pd.read_parquet('labelsBIBtracks.parquet')
recon2d = pd.read_parquet('recon2DBIBtracks.parquet')
recon3d = pd.read_parquet('recon3DBIBtracks.parquet')
print(truth.shape, recon2d.shape, recon3d.shape)

(28127, 13) (28127, 273) (28127, 5460)


In [7]:
array1_list = []
for time_idx in range(20):
    clusterSlice = recon3d.iloc[50].to_numpy().reshape(20,13,21)[time_idx]
    time_label = 200*time_idx + 200
    final_in = reshape_pixelav(clusterSlice, time_label) 
    bit_string = make_bit_string(final_in)
    array = make_array1(bit_string)
    array1_list.append(array)